# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

Nath\*, Mathis\* et al. *Using DeepLabCut for markerless pose estimation during behavior across species*, (under revision).

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

*Note*: Refine a network based after the network was trained on just a few labeled images is illustrated in "Demo-labeledexample-MouseReaching.ipynb". This demo also contains an already labeled data set and is perhaps the best starting point for brand new users.

## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut

In [5]:
task='Tracking' # Enter the name of your experiment Task
experimenter='Pia' # Enter the name of the experimenter
working_dir = '/home/alfomi/work/DLC_DOCKER/example-pia/'
video=[working_dir+'video/2018.10.22_18_01_56.avi'] # Enter the paths of your videos you want to grab frames from.

deeplabcut.create_new_project(task,experimenter,video, working_directory=working_dir,copy_videos=False) #change the working directory to where you want the folders created.

Created "/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/videos"
Created "/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/labeled-data"
Created "/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/training-datasets"
Created "/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/dlc-models"
Creating the symbolic link of the video
Created the symlink of /home/alfomi/work/DLC_DOCKER/example-pia/video/2018.10.22_18_01_56.avi to /home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/videos/2018.10.22_18_01_56.avi
/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/videos/2018.10.22_18_01_56.avi
Generated "/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/config.yaml"

A new project with name Tracking-Pia-2018-12-06 is created at /home/alfomi/work/DLC_DOCKER/example-pia and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once

'/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/config.yaml'

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [6]:
%matplotlib inline
path_config_file = '/home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'automatic','kmeans',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!

36it [00:00, 353.86it/s]

Reading config file successfully...
Extracting frames based on kmeans ...
Kmeans-quantization based extracting of frames from 0.0  seconds to 27.0  seconds.
Extracting and downsampling... 5400  frames from the video.


5400it [00:14, 384.83it/s]


Kmeans clustering ... (this might take a while)

Frames are selected.
You can now label the frames using the function 'label_frames'.


## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [7]:
%gui wx
deeplabcut.label_frames(path_config_file)

Working on folder: 2018.10.22_18_01_56
You can now check the labels, using 'check_labels' before proceeding. Then,  you can use the function 'create_training_dataset' to create the training dataset.


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [8]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Pia.
They are stored in the following folder: /home/alfomi/work/DLC_DOCKER/example-pia/Tracking-Pia-2018-12-06/labeled-data/2018.10.22_18_01_56_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [0]:
deeplabcut.create_training_dataset(path_config_file)

## Start training - If you want to use a CPU, continue. 
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [0]:
deeplabcut.train_network(path_config_file)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [0]:
deeplabcut.evaluate_network(path_config_file)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [0]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [0]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi'])

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [0]:
%gui wx
deeplabcut.refine_labels(path_config_file)

In [0]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [0]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)